In [1]:
# gesture_recognition_live.py

import cv2
import mediapipe as mp
import numpy as np
import pickle

# Load trained model
with open('hand_gesture_model.pkl', 'rb') as f:
    model = pickle.load(f)

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# Extract both-hand landmarks
def extract_both_hand_landmarks(results):
    all_landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                all_landmarks.extend([lm.x, lm.y, lm.z])
    while len(all_landmarks) < 126:
        all_landmarks.extend([0.0] * 3)
    return all_landmarks

cap = cv2.VideoCapture(0)
print("🤖 Hand gesture recognition started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        input_data = extract_both_hand_landmarks(results)
        prediction = model.predict([input_data])[0]

        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.putText(frame, f'Gesture: {prediction}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

🤖 Hand gesture recognition started. Press 'q' to quit.
